In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

1.14.0
Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2


In [2]:
params = ingestor.get_params()

In [3]:
train_data = data.get_train_dataset(params)
parser = data.tristan_parser
train_parsed = train_data.map(functools.partial(parser, params=params))

In [24]:
# for i in train_parsed.take(1):
#     print(i)

In [23]:
for i, j in train_parsed.enumerate():
    l = i
    
print(l)

tf.Tensor(376967, shape=(), dtype=int64)


In [42]:
# set features and labels
features = {'goal': [], 'goal_asl': [], 'thms': [], 'thms_hard_negatives': []}
labels = {'tac_id': []}

# iterate over dataset to extract data into arrays. remove 'take' part to iterate over the entire dataset
for raw_record in train_parsed.take(100):
    fx, lx = raw_record[0], raw_record[1]
    features['goal'].append(fx['goal'])
    features['goal_asl'].append(fx['goal_asl'])
    features['thms'].append(fx['thms'])
    features['thms_hard_negatives'].append(fx['thms_hard_negatives'])
    labels['tac_id'].append(lx['tac_id'])

# instantiate extractor object
ex = extractor2.Extractor(params)

# tokenize goals
temp = ex.tokenize(features['goal'], ex.vocab_table)
#pad all goals to be of length 1000
goal_list = []
for j in range(len(temp)):
        l = len(temp[j])
        h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
        goal_list.append(h)
features['goal_ids'] = goal_list

# tokenize hypotheses. this requires more work since there may be more than one hypothesis
length = len(features['goal'])
features['goal_asl_ids'] = []

for i in range(length):
    temp = ex.tokenize(features['goal_asl'][i], ex.vocab_table)
    #pad all hypotheses to be of length 1000
    hypo_list = []
    for j in range(len(temp)):
        l = len(temp[j])
        h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
        hypo_list.append(h)
    features['goal_asl_ids'].append(hypo_list)

del features['goal']
del features['goal_asl']
del features['thms']
del features['thms_hard_negatives']

# FEATURES
# 'goal_ids': an array of LENGTH OF DATASET containing arrays which are the tokenized goals of length 1000
# below is an example of how to access the entire array as numpy arrays
# print(features['goal_ids'][0].numpy())
# 'goal_asl_ids': this below is an array of LENGTH OF DATASET containing lists of tokenized hypotheses 
# where each hypothesis is of length 1000. below is an example of how to access the first hypothesis 
# from the list of hypotheses corresponding to the first training example
# print(features['goal_asl_ids'][0][0].numpy())

# LABELS
# 'tac_id': array containing tactic ids. below is an example of how to access the first tactic
# print(labels['tac_id'][0].numpy())

In [39]:
tf.shape(features['goal_ids'])

<tf.Tensor: id=3380960, shape=(2,), dtype=int32, numpy=array([   5, 1000], dtype=int32)>

In [46]:
for i in features['goal_asl_ids']:
    print(tf.shape(i))

tf.Tensor([   6 1000], shape=(2,), dtype=int32)
tf.Tensor([   5 1000], shape=(2,), dtype=int32)
tf.Tensor([   7 1000], shape=(2,), dtype=int32)
tf.Tensor([   7 1000], shape=(2,), dtype=int32)
tf.Tensor([   6 1000], shape=(2,), dtype=int32)
tf.Tensor([  53 1000], shape=(2,), dtype=int32)
tf.Tensor([  20 1000], shape=(2,), dtype=int32)
tf.Tensor([0], shape=(1,), dtype=int32)
tf.Tensor([  18 1000], shape=(2,), dtype=int32)
tf.Tensor([   1 1000], shape=(2,), dtype=int32)
tf.Tensor([   4 1000], shape=(2,), dtype=int32)
tf.Tensor([  20 1000], shape=(2,), dtype=int32)
tf.Tensor([0], shape=(1,), dtype=int32)
tf.Tensor([   6 1000], shape=(2,), dtype=int32)
tf.Tensor([  13 1000], shape=(2,), dtype=int32)
tf.Tensor([  16 1000], shape=(2,), dtype=int32)
tf.Tensor([0], shape=(1,), dtype=int32)
tf.Tensor([   9 1000], shape=(2,), dtype=int32)
tf.Tensor([   8 1000], shape=(2,), dtype=int32)
tf.Tensor([   1 1000], shape=(2,), dtype=int32)
tf.Tensor([0], shape=(1,), dtype=int32)
tf.Tensor([   3 1000], s